# Handling Dataset

## Missing Values

In [201]:
import pandas as pd
import numpy as np

예제 데이터 만들기

In [364]:
data = [
    np.random.choice(['A', 'B', 'C'], size=30), 
    np.random.choice(['Male', 'Female'], size=30),
    np.random.randint(100, size=30), 
    np.random.rand(30), 
    np.random.uniform(10, 20, size=30),
    np.random.randint(1000, 20000, size=30),
]

df = pd.DataFrame(data=zip(*data), columns=['col{}'.format(i+1) for i in range(len(data))])
df.head()

,col1,col2,col3,col4,col5,col6
0,B,Male,25,0.649722,16.273599,7190
1,C,Female,51,0.680805,10.767504,6590
2,C,Male,72,0.614635,16.511681,3513
3,B,Female,38,0.503425,12.175674,6006
4,B,Female,68,0.823270,15.165969,13547


예제 데이터 마지막 변수에 결측치 삽입 및 확인

In [365]:
df.iloc[np.random.choice(df.index, size=5, replace=False), -1] = np.nan
idx = df[df.iloc[:,-1].isnull()].index
df[df.iloc[:,-1].isnull()]

,col1,col2,col3,col4,col5,col6
0,B,Male,25,0.649722,16.273599,NaN
11,A,Male,66,0.253154,11.789093,NaN
12,C,Male,70,0.370228,16.158072,NaN
24,C,Female,34,0.231993,12.211881,NaN
29,C,Female,8,0.553463,17.616691,NaN


In [366]:
df.groupby(by='col1')['col6'].agg('mean').to_frame()

,col6
col1,
A,11846.714286
B,13383.555556
C,8586.111111


dataframe.groupby를 사용하여 조건부 평균으로 결측치 처리

In [367]:
df['col6'].fillna(df.groupby(by='col1')['col6'].transform('mean'), inplace=True)
df.iloc[idx,:]

,col1,col2,col3,col4,col5,col6
0,B,Male,25,0.649722,16.273599,13383.555556
11,A,Male,66,0.253154,11.789093,11846.714286
12,C,Male,70,0.370228,16.158072,8586.111111
24,C,Female,34,0.231993,12.211881,8586.111111
29,C,Female,8,0.553463,17.616691,8586.111111


다중 조건으로 처리

In [368]:
df.iloc[np.random.choice(df.index, size=5, replace=False), -1] = np.nan
idx = df[df.iloc[:,-1].isnull()].index
df[df.iloc[:,-1].isnull()]

,col1,col2,col3,col4,col5,col6
2,C,Male,72,0.614635,16.511681,NaN
11,A,Male,66,0.253154,11.789093,NaN
15,C,Male,30,0.239831,12.426514,NaN
28,C,Female,63,0.744728,18.375176,NaN
29,C,Female,8,0.553463,17.616691,NaN


In [369]:
df.groupby(['col1', 'col2'])['col6'].mean().to_frame()

col6
col1 col2                
A    Female  14056.500000
     Male     8900.333333
B    Female  14361.666667
     Male    11916.388889
C    Female   7814.037037
     Male     7442.622222

In [370]:
df['col6'].fillna(df.groupby(['col1', 'col2'])['col6'].transform(np.mean), inplace=True)
df.iloc[idx,:]

,col1,col2,col3,col4,col5,col6
2,C,Male,72,0.614635,16.511681,7442.622222
11,A,Male,66,0.253154,11.789093,8900.333333
15,C,Male,30,0.239831,12.426514,7442.622222
28,C,Female,63,0.744728,18.375176,7814.037037
29,C,Female,8,0.553463,17.616691,7814.037037


범주형 데이터 빈도로 결측치 처리

In [372]:
df.loc[np.random.choice(df.index, 5), 'col2'] = np.nan
idx = df[df['col2'].isnull()].index
df[df['col2'].isnull()]

,col1,col2,col3,col4,col5,col6
0,B,NaN,25,0.649722,16.273599,13383.555556
7,C,NaN,81,0.257888,14.337052,8266.000000
17,C,NaN,60,0.418565,11.572716,5199.000000
26,A,NaN,66,0.715500,15.749490,10065.000000


In [373]:
df.groupby('col1')['col2'].agg(lambda x: x.value_counts().index[0]).to_frame()

,col2
col1,
A,Male
B,Female
C,Male


In [374]:
df['col2'].fillna(df.groupby(by='col1')['col2'].transform(lambda x: x.value_counts().index[0]), inplace=True)
df.iloc[idx,:]

,col1,col2,col3,col4,col5,col6
0,B,Female,25,0.649722,16.273599,13383.555556
7,C,Male,81,0.257888,14.337052,8266.000000
17,C,Male,60,0.418565,11.572716,5199.000000
26,A,Male,66,0.715500,15.749490,10065.000000


결측치가 없는 경우, pd.Series.mode가 빠름

In [340]:
df.groupby('col1')['col2'].agg(pd.Series.mode).to_frame()

,col2
col1,
A,Male
B,Male
C,Female


범주형 변수 합치기

In [341]:
df['new'] = df['col1'].str.cat(df['col2'], sep='+')
df.head()

,col1,col2,col3,col4,col5,col6,new
0,A,Male,73,0.654390,12.484973,5071.0,A+Male
1,A,Female,82,0.917650,10.643406,14330.0,A+Female
2,C,Female,51,0.568164,17.922425,11395.0,C+Female
3,C,Female,23,0.741450,19.927032,11395.0,C+Female
4,B,Male,85,0.921404,19.628258,19889.0,B+Male


object 형태를 category 형태로 변환하기: 데이터셋의 용량을 현저히 줄일 수 있다.

In [375]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 6 columns):
col1    30 non-null object
col2    30 non-null object
col3    30 non-null int64
col4    30 non-null float64
col5    30 non-null float64
col6    30 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 1.5+ KB


In [376]:
cols = df.select_dtypes(include='object').columns
print(cols)

Index(['col1', 'col2'], dtype='object')


In [377]:
df[cols] = df[cols].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 6 columns):
col1    30 non-null category
col2    30 non-null category
col3    30 non-null int64
col4    30 non-null float64
col5    30 non-null float64
col6    30 non-null float64
dtypes: category(2), float64(3), int64(1)
memory usage: 1.3 KB


In [346]:
df['col2'].cat.categories

Index(['Female', 'Male'], dtype='object')

In [350]:
df['col2'].cat.codes[:5]

0    1
1    0
2    0
3    0
4    1
dtype: int8

pd.get_dummies로 더미변수 또는 원-핫 인코딩 만들기, scikit-learn의 one-hot보다 빠르다고 함

In [378]:
dummy = pd.get_dummies(df['col2'])
pd.concat([df, dummy], axis=1).head()

,col1,col2,col3,col4,col5,col6,Female,Male
0,B,Female,25,0.649722,16.273599,13383.555556,1,0
1,C,Female,51,0.680805,10.767504,6590.000000,1,0
2,C,Male,72,0.614635,16.511681,7442.622222,0,1
3,B,Female,38,0.503425,12.175674,6006.000000,1,0
4,B,Female,68,0.823270,15.165969,13547.000000,1,0


,Col0,Col1,Col2,Col3,Col4
0,A,X,5,0.919447,NaN
1,B,Y,58,0.035553,NaN
2,A,Y,62,0.496671,17.127362
3,C,X,17,0.601528,17.941758
4,C,X,24,0.950626,NaN


In [120]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

import re 

train['Title'] = train['Name'].apply(lambda x: re.search('([A-Za-z]+)\.', x).group(1))
test['Title'] = test['Name'].apply(lambda x: re.search('([A-Za-z]+)\.', x).group(1))
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,Mr
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,Mrs
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,Mr
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,Mr
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,Mrs


In [121]:
train[(train['Title'].isin(['Mr', 'Miss'])) & (train['Age'].isnull())].head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,Mr
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S,Mr
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C,Mr
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q,Miss
29,30,0,3,"Todoroff, Mr. Lalio",male,NaN,0,0,349216,7.8958,NaN,S,Mr


In [122]:
train['Age'].fillna(train.groupby('Title')['Age'].transform('mean'), inplace=True)
train.iloc[[5,17,26,28,29], :]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
5,6,0,3,"Moran, Mr. James",male,32.368090,0,0,330877,8.4583,NaN,Q,Mr
17,18,1,2,"Williams, Mr. Charles Eugene",male,32.368090,0,0,244373,13.0000,NaN,S,Mr
26,27,0,3,"Emir, Mr. Farred Chehab",male,32.368090,0,0,2631,7.2250,NaN,C,Mr
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,21.773973,0,0,330959,7.8792,NaN,Q,Miss
29,30,0,3,"Todoroff, Mr. Lalio",male,32.368090,0,0,349216,7.8958,NaN,S,Mr


In [123]:
test[(test['Title'].isin(['Mr', 'Miss'])) & (test['Age'].isnull())].head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
10,902,3,"Ilieff, Mr. Ylio",male,NaN,0,0,349220,7.8958,NaN,S,Mr
29,921,3,"Samaan, Mr. Elias",male,NaN,2,0,2662,21.6792,NaN,C,Mr
36,928,3,"Roth, Miss. Sarah A",female,NaN,0,0,342712,8.0500,NaN,S,Miss
39,931,3,"Hee, Mr. Ling",male,NaN,0,0,1601,56.4958,NaN,S,Mr
41,933,1,"Franklin, Mr. Thomas Parham",male,NaN,0,0,113778,26.5500,D34,S,Mr


In [124]:
vals = train.groupby('Title')['Age'].agg('mean').to_dict()
vals

{'Capt': 70.0,
 'Col': 58.0,
 'Countess': 33.0,
 'Don': 40.0,
 'Dr': 42.0,
 'Jonkheer': 38.0,
 'Lady': 48.0,
 'Major': 48.5,
 'Master': 4.574166666666667,
 'Miss': 21.773972602739743,
 'Mlle': 24.0,
 'Mme': 24.0,
 'Mr': 32.36809045226126,
 'Mrs': 35.898148148148145,
 'Ms': 28.0,
 'Rev': 43.166666666666664,
 'Sir': 49.0}

In [125]:
test[(test['Title'].isin(['Mr', 'Miss'])) & (test['Age'].isnull())].head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
10,902,3,"Ilieff, Mr. Ylio",male,NaN,0,0,349220,7.8958,NaN,S,Mr
29,921,3,"Samaan, Mr. Elias",male,NaN,2,0,2662,21.6792,NaN,C,Mr
36,928,3,"Roth, Miss. Sarah A",female,NaN,0,0,342712,8.0500,NaN,S,Miss
39,931,3,"Hee, Mr. Ling",male,NaN,0,0,1601,56.4958,NaN,S,Mr
41,933,1,"Franklin, Mr. Thomas Parham",male,NaN,0,0,113778,26.5500,D34,S,Mr


In [128]:
test.loc[test['Age'].isnull(), 'Age'] = test[test['Age'].isnull()]['Title'].replace(vals)
test.iloc[[10,29,36,39,41], :]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
10,902,3,"Ilieff, Mr. Ylio",male,32.3681,0,0,349220,7.8958,NaN,S,Mr
29,921,3,"Samaan, Mr. Elias",male,32.3681,2,0,2662,21.6792,NaN,C,Mr
36,928,3,"Roth, Miss. Sarah A",female,21.774,0,0,342712,8.0500,NaN,S,Miss
39,931,3,"Hee, Mr. Ling",male,32.3681,0,0,1601,56.4958,NaN,S,Mr
41,933,1,"Franklin, Mr. Thomas Parham",male,32.3681,0,0,113778,26.5500,D34,S,Mr


In [129]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,Mr
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47,1,0,363272,7.0000,NaN,S,Mrs
2,894,2,"Myles, Mr. Thomas Francis",male,62,0,0,240276,9.6875,NaN,Q,Mr
3,895,3,"Wirz, Mr. Albert",male,27,0,0,315154,8.6625,NaN,S,Mr
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22,1,1,3101298,12.2875,NaN,S,Mrs


In [409]:
df['발생형태3'].value_counts()

Fall from height    4527
Collision           1845
Rollover            1674
Falling objects     1317
Stucked              731
Collapse             418
Other                351
Electrical           195
Fire                 167
Name: 발생형태3, dtype: int64

In [425]:
df.isnull().sum()

0

In [8]:
df.reset_index(drop=True, inplace=True)

In [423]:
# 추락사고인데 추락높이가 없는 값 전처리
df['추락높이'] = df['추락높이'][(df['발생형태3'] == 'Fall from height') & (df['추락높이'].isnull())].fillna('Unknown')

clean['추락높이'].replace({None: 'Not fall accident'}, inplace=True)
clean['개인보호장비착용'].replace({None: 'Not wearing'}, inplace=True)

In [448]:
df.isnull().sum()

first_name       1
last_name        1
age              1
sex              1
preTestScore     2
postTestScore    2
dtype: int64

In [ ]:
df['근무기간'].fillna(df.groupby('연령')['근무기간'].transform(lambda x: x.value_counts().index[0]))

In [12]:
df.groupby('연령')['근무기간'].agg(lambda x: x.value_counts().index[0])

연령
18 - 19    Under 1 month
20 - 24    Under 1 month
25 - 29    Under 1 month
30 - 34    Under 1 month
35 - 39    Under 1 month
40 - 44    Under 1 month
45 - 49    Under 1 month
50 - 54    Under 1 month
55 - 59    Under 1 month
60 - 64    Under 1 month
< 18       Under 1 month
≥ 65       Under 1 month
Name: 근무기간, dtype: object

In [497]:
df.pivot_table(index='연령', columns='근무기간', aggfunc='count', fill_value=0).loc[:,'가해물1']

근무기간,1 - 2 months,1 - 2 years,10 - 20 years,2 - 3 months,2 - 3 years,3 - 4 months,3 - 4 years,4 - 5 months,4 - 5 years,5 - 10 years,5 - 6 months,6 - 12 months,Over 20 years,Under 1 month
연령,,,,,,,,,,,,,,
18 - 19,1,0,0,0,1,0,0,0,0,0,0,0,0,5
20 - 24,7,3,2,5,3,5,1,0,1,0,0,2,0,18
25 - 29,15,22,8,8,12,10,15,6,5,18,4,19,0,54
30 - 34,17,47,23,6,29,12,27,12,12,54,10,25,4,122
35 - 39,50,51,133,21,33,18,32,11,15,110,7,33,10,195
40 - 44,75,54,262,54,32,37,54,16,26,138,14,52,79,377
45 - 49,134,78,298,55,61,37,46,17,28,161,20,73,224,571
50 - 54,171,81,304,82,46,78,65,25,30,172,16,78,382,719
55 - 59,138,59,234,69,41,47,41,28,30,126,18,70,343,686


In [13]:
# from scipy import stats

# df.groupby(['연령', '근무기간']).agg(lambda x: stats.mode(x)[0][0])


In [292]:
# df = df[df['재해유형'] == 'Nonfatal'].copy()
# print(df.shape)

In [293]:
# # df['근로손실일수'].replace({'≤ 28': '≤ 90', '29 - 90': '≤ 90'}, inplace=True)
# df['근로손실일수'].replace({'≤ 28': '≤ 180', '29 - 90': '≤ 180', '91 - 180': '≤ 180'}, inplace=True)
# df['근로손실일수'].value_counts()

In [280]:
df.pivot_table(values='규모', index='연령', columns='근로손실일수', aggfunc='count')

근로손실일수,≤ 180,≥ 181
연령,,
18 - 19,2,5
20 - 24,24,27
25 - 29,114,91
30 - 34,235,183
35 - 39,449,308
40 - 44,759,554
45 - 49,1056,823
50 - 54,1292,1042
55 - 59,1029,981


In [57]:
df.pivot_table(values='연령', index='고용형태', columns='근로손실일수', aggfunc='count')

근로손실일수,29 - 90,91 - 180,≤ 28,≥ 181
고용형태,,,,
Daily,2436.0,1430.0,631.0,1331.0
Full-time,320.0,221.0,72.0,154.0
Part-time,2.0,1.0,NaN,NaN
Temporary,90.0,62.0,18.0,120.0


In [58]:
df.pivot_table(values='연령', index='고용형태', columns='근로손실일수', aggfunc='count')/df['근로손실일수'].value_counts()

,29 - 90,91 - 180,≤ 28,≥ 181
고용형태,,,,
Daily,0.855337,0.834306,0.875173,0.829283
Full-time,0.112360,0.128938,0.099861,0.095950
Part-time,0.000702,0.000583,NaN,NaN
Temporary,0.031601,0.036173,0.024965,0.074766


In [59]:
df.pivot_table(values='연령', index='질병부위3', columns='근로손실일수', aggfunc='count')

근로손실일수,29 - 90,91 - 180,≤ 28,≥ 181
질병부위3,,,,
Head,300,162,157,152
Lower limb,758,512,158,517
Multiple body parts,52,42,10,80
Neck,57,26,15,24
Soma(body),748,400,126,362
Upper limb,927,569,253,465
Whole body,6,3,2,5


In [60]:
df.pivot_table(values='연령', index='질병부위3', columns='근로손실일수', aggfunc='count')/df['근로손실일수'].value_counts()

,29 - 90,91 - 180,≤ 28,≥ 181
질병부위3,,,,
Head,0.105337,0.094516,0.217753,0.094704
Lower limb,0.266152,0.298716,0.219140,0.322118
Multiple body parts,0.018258,0.024504,0.013870,0.049844
Neck,0.020014,0.015169,0.020804,0.014953
Soma(body),0.262640,0.233372,0.174757,0.225545
Upper limb,0.325492,0.331972,0.350902,0.289720
Whole body,0.002107,0.001750,0.002774,0.003115


In [61]:
df.pivot_table(values='연령', index='기인물3', columns='근로손실일수', aggfunc='count')[df.pivot_table(values='연령', index='기인물3', columns='근로손실일수', aggfunc='count') > 10].dropna()

근로손실일수,29 - 90,91 - 180,≤ 28,≥ 181
기인물3,,,,
Components or accessories of buildings and structures,65.0,29.0,16.0,26.0
Edge or opening,89.0,69.0,12.0,67.0
Equipment and machinery parts or accessories,56.0,35.0,15.0,33.0
Floor or ground,220.0,129.0,60.0,106.0
Form or support,232.0,107.0,77.0,110.0
Hand tools,113.0,56.0,44.0,44.0
Manufacturing machines,131.0,91.0,31.0,72.0
Materials,327.0,178.0,112.0,148.0
Nonmetallic products,79.0,44.0,25.0,22.0


In [62]:
df.pivot_table(values='연령', index='규모', columns='근로손실일수', aggfunc='count').sum()

근로손실일수
29 - 90     2848
91 - 180    1714
≤ 28         721
≥ 181       1605
dtype: int64

In [63]:
df.pivot_table(values='연령', index='공사금액', columns='근로손실일수', aggfunc='count').sum()

근로손실일수
29 - 90     2848
91 - 180    1714
≤ 28         721
≥ 181       1605
dtype: int64

In [64]:
df.groupby(by='근로손실일수').agg('count').T

근로손실일수,29 - 90,91 - 180,≤ 28,≥ 181
연령,2848,1714,721,1605
규모,2848,1714,721,1605
공사금액,2848,1714,721,1605
직업2,2848,1714,721,1605
고용형태,2848,1714,721,1605
근무기간,2848,1714,721,1605
재해유형,2848,1714,721,1605
질병명3,2848,1714,721,1605
질병부위3,2848,1714,721,1605
발생형태3,2848,1714,721,1605
